# 2. Data Selection

> “The quality of a composite indicator is directly linked to the quality of the variables selected.” - Gotten from the OECD Handbook on Composite Indicators

In this notebook, I will select the data that will be used to calculate the CSIAI and also justifications where needed.

## 2.1. Statistical Quality Principles

There are principles that an indicator must satisfy **all seven** or be rejected:

1. **Relevance** — captures something the theoretical framework says matters.  
2. **Accuracy** — sourced from audited statements and raw market data.  
3. **Timeliness** — updates at least quarterly; daily preferred in my case as discussed with **Dr. John Loane** and it is also the frequency of the CSIAI.  
4. **Accessibility** — free via `yfinance`.  
5. **Interpretability** — unit and direction are intuitive.  
6. **Comparability** — works across all sectors.  
7. **Coherence** — definitions do not conflict with other metrics.

If any of the indicators fail to meet these principles then I will exclude them or I can use a proxy with proper justification.

## 2.2. Defining the Dataset

### 2.2.1. Why Russell 3000?

* it covers 98 % of U.S. market capitalisation and is the most widely used benchmark for U.S. equities.
* It is a broad index that includes large, mid, and small-cap stocks.
* The list is public and can be found on [Wikipedia](https://en.wikipedia.org/wiki/Russell_3000_Index).
* The index is reconstituted annually, which means that it is updated regularly to reflect changes in the market.

### 2.2.2. Why require 24 months of price history (start 2023-01-01)?

The Risk metrics (beta, Sharpe ratio, maximum draw-down) need at least one full market cycle and in the Handbook in Section 3 it recommends “adequate observational base”.  Two years provides:
* ≈ 500 trading days: This is a reliable number of trading days to estimate volatility.
* IPOs - Initial Public Offerings - younger than 6 months are excluded to avoid data sparsity.

### 2.2.3. Why filter on **average 30-day volume ≥ 50 000 shares**?

* Ensures **Liquidity & Trading** sub-index is not dominated by stocks that are thinly traded.
* 50,000 shares/day is a common threshold for liquidity in the finance literature taking for example in this case the work of **Gao & Ongena (2021)** and this threshold keeps ≈ 80 % of the Russell 3000 index.
* This threshold also reduces the estimation error in bid-ask spread calculations, which is about how the market is functioning and how much it costs to trade a stock.

### 2.2.4  Reliable ticker source

The iShares Russell 3000 ETF (ticker **IWV**) publishes its full holdings every night as a CSV file.

Advantages:
* **Authoritative** — the fund must hold every Russell 3000 constituent.  
* **Timely** — file refreshes after each U.S. trading day.  
* **Stable URL** — the `.ajax` endpoint is version agnostic  which means it will not change.
* **Free** — no login or API key required.
* **Clean “Holdings” sheet** — tickers are listed in a single column.

We skip the first **seven** metadata rows, treat row 8 as the header, select the **“Ticker”** column, and drop empties, dashes, or placeholders - All these will be done dynamically in the code.

* **Local CSV fallback**  
If the network call fails or you simply prefer, drop the current file you downloaded manually at `data/input/IWV_holdings.csv`. The loader will auto-detect it and skip the web request.

In [1]:
import pandas as pd, requests, io, os, datetime as dt, yfinance as yf
from tqdm import tqdm
import warnings, pathlib
import csv

warnings.filterwarnings("ignore")
# Define the base data directory
DATA_DIR = pathlib.Path("..") / "data"

# Ensure the data directory exists
DATA_DIR.mkdir(parents=True, exist_ok=True)

INPUT_DIR = DATA_DIR / "input"
PROCESSED_DIR = DATA_DIR / "processed"

# Ensure folders exist
INPUT_DIR.mkdir(parents=True, exist_ok=True)
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

LOCAL_FILE = INPUT_DIR / "IWV_holdings.csv"

REMOTE_URL = ("https://www.ishares.com/us/products/239714/ishares-russell-3000-etf/1467271812596.ajax?fileType=csv&fileName=IWV_holdings&dataType=fund")

def load_iwv_csv() -> bytes:
    "Return CSV bytes local first, else remote download and cache."
    if LOCAL_FILE.exists() and LOCAL_FILE.stat().st_size > 0:
        print("Using local IWV CSV:", LOCAL_FILE)
        return LOCAL_FILE.read_bytes()
    print("Downloading IWV holdings CSV …")
    r = requests.get(REMOTE_URL, timeout=30)
    r.raise_for_status()
    LOCAL_FILE.write_bytes(r.content)
    print("Saved snapshot to:", LOCAL_FILE)
    return r.content

def extract_tickers(csv_bytes: bytes) -> list[str]:
    """
    Parses the CSV content to extract ticker symbols.
    Handles metadata at the beginning and footnotes at the end.
    """
    # Decode the bytes to a string
    csv_text = csv_bytes.decode('utf-8', errors='ignore')
    lines = csv_text.splitlines()

    # Identify the header row
    header_line_index = None
    for i, line in enumerate(lines):
        if 'Ticker' in line:
            header_line_index = i
            break

    if header_line_index is None:
        raise ValueError("Ticker header not found in the CSV file.")

    # Read the data starting from the header
    data_lines = lines[header_line_index:]

    # Stop reading when an empty line is encountered
    for j, line in enumerate(data_lines):
        if not line.strip():
            data_lines = data_lines[:j]
            break

    # Create a DataFrame from the data lines
    data_str = '\n'.join(data_lines)
    df = pd.read_csv(io.StringIO(data_str))

    # Clean and extract ticker symbols
    tickers = (df['Ticker'].astype(str).str.strip().replace({'': pd.NA, '-': pd.NA}).dropna().str.replace(r'\.', '-', regex=True).unique().tolist())

    return tickers

# to run the loader
csv_bytes = load_iwv_csv()
tickers = extract_tickers(csv_bytes)
print(f"Fetched {len(tickers)} tickers from IWV holdings")

# The parameters to filter the tickers in the yfinance
START, END = "2023-01-01", dt.date.today().isoformat()
# shares/day
VOL_THRESHOLD = 50_000

# Liquidity filter using a 40 day window
prices = yf.download(" ".join(tickers), start=dt.date.fromisoformat(END) - dt.timedelta(days=40), end=END, group_by="ticker", threads=True, progress=False)

liquid = []
for t in tickers:
    try:
        if prices[t]["Volume"].tail(30).mean() >= VOL_THRESHOLD:
            liquid.append(t)
    except KeyError:
        pass

print(f"Universe size after liquidity filter: {len(liquid)}")
universe = pd.DataFrame({"ticker": liquid})
universe.to_parquet(PROCESSED_DIR / "universe.parquet", index=False)

/Users/patrick/Documents/Final year/csiai-project/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Using local IWV CSV: ../data/input/IWV_holdings.csv
Fetched 2656 tickers from IWV holdings
YF.download() has changed argument auto_adjust default to True



21 Failed downloads:
['INH', 'CLSKW']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-03-27 -> 2025-05-06)')
['LGFB', 'CWENA']: HTTPError('HTTP Error 401: ')
['BFA', 'P5N994', 'ESM5', 'BFB', 'GTXI', 'BRKB', 'MOGA', 'METCV', 'ADRO', 'RTYM5', 'LENB', 'LGFA', 'UHALB', 'HEIA', 'GEFB', 'MSFUT', 'XTSLA']: YFTzMissingError('possibly delisted; no timezone found')


Universe size after liquidity filter: 2563


## 2.3. The Indicator List

Using the theoretical framework from the Handbook, I will select the indicators that will be used to calculate the CSIAI.  The indicators are grouped into five sub-indices: **Financial Strength**, **Growth Potential**, **Market Performance**, **Risk & Volatility**, and **Liquidity & Trading**.
The indicators are selected based on their relevance to the theoretical framework and their statistical quality.  The indicators are also selected based on their availability in the `yfinance` library, which is used to retrieve the data.

| Dim. | Indicator | yfinance field / derivation | Justification |
|------|-----------|-----------------------------|---------------|
| **Financial Strength** | Return on Equity | `info['returnOnEquity']` | Profitability per unit equity. |
| | Debt-to-Equity | `info['debtToEquity']` | Capital structure risk. |
| | Current Ratio | `info['currentRatio']` | Short-term solvency. |
| | Quick Ratio | `info['quickRatio']` | Acid-test liquidity. |
| | Oper. Cash Flow | `info['operatingCashflow']` | Cash backing earnings. |
| **Growth Potential** | Revenue Growth | YoY revenue trend | Top-line expansion. |
| | Earnings Growth | YoY EPS trend | Bottom-line expansion. |
| | Operating Margin | `info['operatingMargins']` | Efficiency scaling. |
| | Gross Margin | `info['grossMargins']` | Core pricing power. |
| | Analyst Rating | `info['recommendationMean']` | External sentiment. |
| **Market Performance** | P/E Ratio | `info['trailingPE']` | Classic valuation. |
| | EPS | `info['trailingEps']` | Profit per share. |
| | P/B Ratio | `info['priceToBook']` | Asset-based valuation. |
| | Dividend Yield | `info['dividendYield']` | Income return. |
| | Market Cap | `info['marketCap']` | Size proxy. |
| **Risk & Volatility** | 30-d Hist. Volatility | `std(returns)` | Short-term risk. |
| | Beta | `info['beta']` | Market sensitivity. |
| | Sharpe Ratio | excess return / vol | Risk-adjusted perf. |
| | Max Draw-down | roll-min formula | Tail risk. |
| | Std Dev Returns | `std(returns)` | Dispersion measure. |
| **Liquidity & Trading** | Avg 30-d Volume | rolling mean | Depth of book. |
| | Bid-Ask Spread* | `(High−Low)/Mid` | Transaction cost proxy. |
| | Volume Growth | pct_change Volume | Activity momentum. |
| | Shares Outstanding | `info['sharesOutstanding']` | Supply side. |
| | Float Shares | `info['floatShares']` | Free float liquidity. |


`*` *Bid-Ask Spread* is a proxy for transaction costs.  The formula is `(High−Low)/Mid` where `Mid` is the average of the high and low prices.  This is a common proxy in finance papers, but it is not perfect.  The bid-ask spread can be affected by many factors, including market conditions, liquidity, and trading volume.

In [2]:
import json, os, datetime as dt
import pathlib

DOCS_DIR = pathlib.Path("..") / "docs"

# Ensure the docs directory exists
DOCS_DIR.mkdir(parents=True, exist_ok=True)

METADATA_DIR = DOCS_DIR / "metadata"
# Ensure the metadata directory exists
METADATA_DIR.mkdir(parents=True, exist_ok=True)

base = "https://query2.finance.yahoo.com/v10/finance/quoteSummary"

today = dt.date.today().isoformat()

def yurl(ticker, module):
    return f"{base}/{ticker}?modules={module}"

var_dict = {
    # Financial Strength
    "roe": {
        "dim": "financial_strength",
        "definition": "Net income divided by total shareholder equity.",
        "unit": "%",
        "source_url": yurl("<ticker>", "defaultKeyStatistics"),
        "refresh": "quarterly",
        "last_updated": today
    },
    "debt_to_equity": {
        "dim": "financial_strength",
        "definition": "Total debt divided by shareholder equity.",
        "unit": "%",
        "source_url": yurl("<ticker>", "defaultKeyStatistics"),
        "refresh": "quarterly",
        "last_updated": today
    },
    "current_ratio": {
        "dim": "financial_strength",
        "definition": "Current assets divided by current liabilities.",
        "unit": "ratio",
        "source_url": yurl("<ticker>", "financialData"),
        "refresh": "quarterly",
        "last_updated": today
    },
    "quick_ratio": {
        "dim": "financial_strength",
        "definition": "Liquid assets divided by current liabilities.",
        "unit": "ratio",
        "source_url": yurl("<ticker>", "financialData"),
        "refresh": "quarterly",
        "last_updated": today
    },
    "oper_cash_flow": {
        "dim": "financial_strength",
        "definition": "Cash generated by core operations.",
        "unit": "USD",
        "source_url": yurl("<ticker>", "cashflowStatementHistoryQuarterly"),
        "refresh": "quarterly",
        "last_updated": today
    },

    # Growth Potential
    "revenue_growth": {
        "dim": "growth_potential",
        "definition": "Year-over-year percentage change in revenue.",
        "unit": "%",
        "source_url": yurl("<ticker>", "incomeStatementHistoryQuarterly"),
        "refresh": "quarterly",
        "last_updated": today
    },
    "earnings_growth": {
        "dim": "growth_potential",
        "definition": "Year-over-year percentage change in EPS.",
        "unit": "%",
        "source_url": yurl("<ticker>", "earningsTrend"),
        "refresh": "quarterly",
        "last_updated": today
    },
    "operating_margin": {
        "dim": "growth_potential",
        "definition": "Operating income divided by revenue.",
        "unit": "%",
        "source_url": yurl("<ticker>", "financialData"),
        "refresh": "quarterly",
        "last_updated": today
    },
    "gross_margin": {
        "dim": "growth_potential",
        "definition": "Gross profit divided by revenue.",
        "unit": "%",
        "source_url": yurl("<ticker>", "financialData"),
        "refresh": "quarterly",
        "last_updated": today
    },
    "analyst_rating": {
        "dim": "growth_potential",
        "definition": "Wall-Street consensus rating (1 = Strong Buy, 5 = Sell).",
        "unit": "score",
        "source_url": yurl("<ticker>", "recommendationTrend"),
        "refresh": "weekly",
        "last_updated": today
    },

    # Market Performance
    "pe_ratio": {
        "dim": "market_performance",
        "definition": "Share price divided by trailing twelve-month EPS.",
        "unit": "ratio",
        "source_url": yurl("<ticker>", "summaryDetail"),
        "refresh": "daily",
        "last_updated": today
    },
    "eps": {
        "dim": "market_performance",
        "definition": "Trailing twelve-month earnings per share.",
        "unit": "USD",
        "source_url": yurl("<ticker>", "defaultKeyStatistics"),
        "refresh": "quarterly",
        "last_updated": today
    },
    "pb_ratio": {
        "dim": "market_performance",
        "definition": "Share price divided by book value per share.",
        "unit": "ratio",
        "source_url": yurl("<ticker>", "defaultKeyStatistics"),
        "refresh": "daily",
        "last_updated": today
    },
    "dividend_yield": {
        "dim": "market_performance",
        "definition": "Annual dividend per share divided by share price.",
        "unit": "%",
        "source_url": yurl("<ticker>", "summaryDetail"),
        "refresh": "daily",
        "last_updated": today
    },
    "market_cap": {
        "dim": "market_performance",
        "definition": "Share price multiplied by shares outstanding.",
        "unit": "USD",
        "source_url": yurl("<ticker>", "price"),
        "refresh": "daily",
        "last_updated": today
    },

    # Risk & Volatility
    "hist_volatility": {
        "dim": "risk_volatility",
        "definition": "30-day annualised standard deviation of returns.",
        "unit": "%",
        "source_url": "price_history",
        "refresh": "daily",
        "last_updated": today
    },
    "beta": {
        "dim": "risk_volatility",
        "definition": "Covariance with S&P 500 returns divided by variance of S&P 500.",
        "unit": "ratio",
        "source_url": yurl("<ticker>", "defaultKeyStatistics"),
        "refresh": "daily",
        "last_updated": today
    },
    "sharpe_ratio": {
        "dim": "risk_volatility",
        "definition": "Excess return divided by volatility.",
        "unit": "ratio",
        "source_url": "price_history",
        "refresh": "daily",
        "last_updated": today
    },
    "max_drawdown": {
        "dim": "risk_volatility",
        "definition": "Largest peak-to-trough drop over last 12 months.",
        "unit": "%",
        "source_url": "price_history",
        "refresh": "daily",
        "last_updated": today
    },
    "stddev_returns": {
        "dim": "risk_volatility",
        "definition": "Standard deviation of daily returns (252-day window).",
        "unit": "%",
        "source_url": "price_history",
        "refresh": "daily",
        "last_updated": today
    },

    # Liquidity & Trading
    "avg_volume_30d": {
        "dim": "liquidity_trading",
        "definition": "30-day moving average of daily volume.",
        "unit": "shares",
        "source_url": "price_history",
        "refresh": "daily",
        "last_updated": today
    },
    "bid_ask_spread": {
        "dim": "liquidity_trading",
        "definition": "Proxy spread: (High − Low) ÷ Mid price.",
        "unit": "%",
        "source_url": "price_history",
        "refresh": "daily",
        "last_updated": today
    },
    "volume_growth": {
        "dim": "liquidity_trading",
        "definition": "Month-over-month percentage change in volume.",
        "unit": "%",
        "source_url": "price_history",
        "refresh": "daily",
        "last_updated": today
    },
    "shares_outstanding": {
        "dim": "liquidity_trading",
        "definition": "Total shares issued by the company.",
        "unit": "shares",
        "source_url": yurl("<ticker>", "defaultKeyStatistics"),
        "refresh": "quarterly",
        "last_updated": today
    },
    "float_shares": {
        "dim": "liquidity_trading",
        "definition": "Shares available for public trading (ex-insiders).",
        "unit": "shares",
        "source_url": yurl("<ticker>", "defaultKeyStatistics"),
        "refresh": "quarterly",
        "last_updated": today
    }
}

with open(METADATA_DIR / "variable_dictionary.json", "w") as fp:
    json.dump(var_dict, fp, indent=2)

print("variable_dictionary.json written with full metadata.")

variable_dictionary.json written with full metadata.


## 2.4. Data Extraction Scripts

This script will download the Russell 3000 tickers and then download the data for each ticker.  The script will also cache the data to avoid repeated API hits.  The script will be run in a CI job that runs every night (My future plan).

* Fundamentals refresh **weekly** (because they change slowly).  
* Prices refresh **nightly** (CI job).  
* Each ticker gets its own file for modular updates.

In [3]:
import joblib, time, random, os, numpy as np, pandas as pd, datetime as dt
from pathlib import Path
import yfinance as yf
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

# Base directory setup
DATA_DIR = Path("..") / "data"
RAW_DIR = DATA_DIR / "raw"
PRICE_DIR = RAW_DIR / "prices"

TODAY = dt.date.today()
START = "2023-01-01"  # start of the year
END = TODAY - dt.timedelta(days=1)  # yesterday
MAX_WORKERS = 6   # Number of threads to use for downloading data

# Create directories if they don't exist
PRICE_DIR.mkdir(parents=True, exist_ok=True)

# Helper function to check if the info file needs to be refreshed for Quarterly data
def need_info_refresh(path: Path, days=90) -> bool:
    return not path.exists() or (TODAY - dt.date.fromtimestamp(path.stat().st_mtime)).days > days

def save_info(ticker: str):
    dest = RAW_DIR / f"info_{ticker}.pkl"
    if need_info_refresh(dest):
        try:
            joblib.dump(yf.Ticker(ticker).info, dest)
            time.sleep(0.2 + random.random() * 0.1)
        except Exception as e:
            return f"info {ticker}: {e}"
    return None

# Helper function to check if the price data file needs to be refreshed daily
def save_price_delta(ticker: str):
    dest = PRICE_DIR / f"{ticker}.parquet"
    try:
        if dest.exists():
            df_old = pd.read_parquet(dest)
            last_day = df_old.index.max().date()
            start_dt = last_day + dt.timedelta(days=1)
        else:
            start_dt = dt.date.fromisoformat(START)

        end_dt = END
        if start_dt > end_dt:
            return None  # no update needed

        df_new = yf.download(ticker, start=start_dt, end=end_dt + dt.timedelta(days=1), progress=False)
        if df_new.empty:
            return None

        if dest.exists():
            df_old = pd.read_parquet(dest)
            df_all = pd.concat([df_old, df_new[~df_new.index.isin(df_old.index)]])
        else:
            df_all = df_new

        df_all.to_parquet(dest)
    except Exception as e:
        return f"price {ticker}: {e}"
    return None

# thread pool to download the data
errors = []
with ThreadPoolExecutor(max_workers=MAX_WORKERS) as tp:
    futures = {tp.submit(save_info, t): t for t in universe["ticker"]}
    for f in tqdm(as_completed(futures), total=len(futures), desc="Fundamentals"):
        err = f.result()
        if err: errors.append(err)

    futures = {tp.submit(save_price_delta, t): t for t in universe["ticker"]}
    for f in tqdm(as_completed(futures), total=len(futures), desc="Prices"):
        err = f.result()
        if err: errors.append(err)

if errors:
    print("Completed with some errors:")
    for e in errors[:10]:
        print("   ", e)

Prices: 100%|██████████| 2563/2563 [00:01<00:00, 1531.20it/s]


## 2.5  Coverage Diagnostics

### 2.5.1  Why **coverage ≥ 90 %**?

* In the OECD Handbook PART 2 section 3, it is recommended to drop indicators with “large data gaps” to avoid heavy imputation bias.
* In finance, missing fundamentals can often show other things like delistings or delayed filings.
* I will cap it at 90% to make sure that the indicator is still relevant and that it is not too sparse.

In [4]:
import pandas as pd, numpy as np, joblib, glob
from pathlib import Path

# Define project paths
DATA_DIR = Path("..") / "data"
RAW_DIR = DATA_DIR / "raw"
ARTIFACT_DIR = Path("..") / "artifacts"
ARTIFACT_DIR.mkdir(parents=True, exist_ok=True)

infos = [joblib.load(f) for f in glob.glob(str(RAW_DIR / "info_*.pkl"))]
df_info = pd.DataFrame(infos)

# Compute non-missing value coverage per indicator
coverage = df_info.notna().mean().sort_values(ascending=False)

coverage.to_csv(ARTIFACT_DIR / "coverage.csv")

# Identify low coverage indicators
low_cov = coverage[coverage < 0.90]

print("Indicators with < 90% coverage:\n", low_cov)

Indicators with < 90% coverage:
 ebitda                          0.891533
enterpriseToEbitda              0.886071
displayName                     0.881389
debtToEquity                    0.862661
freeCashflow                    0.862271
recommendationMean              0.814670
averageAnalystRating            0.814670
trailingPE                      0.687476
earningsQuarterlyGrowth         0.597347
dividendDate                    0.595396
lastDividendValue               0.595006
lastDividendDate                0.595006
exDividendDate                  0.591494
earningsGrowth                  0.588373
dividendYield                   0.518143
dividendRate                    0.517753
address2                        0.490051
fiveYearAvgDividendYield        0.486149
lastSplitDate                   0.467811
lastSplitFactor                 0.467811
irWebsite                       0.376512
trailingPegRatio                0.297698
fax                             0.193133
ipoExpectedDate         

## 2.6. Indicator Refinement Based on Coverage

In Section 2.5, I diagnosed missing data across all available financial indicators and identified some metrics with **coverage below the OECD-recommended threshold of 90%**. The low availability 
might actually pose risk on the final composite index.

### Updated Strategy

To address this, I will do the following:

1. **Expand the indicator set from 25 to ~35 variables** to improve conceptual richness while maintaining high coverage.
2. **Drop indicators** below 80% coverage. I will allow the debt-to-equity ratio and recommendation mean to remain as they are conceptually important.
3. **Add substitute indicators** with high availability (≥90%) and low correlational overlap.

| Dim. | Indicator | yfinance field / derivation | Justification |
|------|-----------|-----------------------------|---------------|
| **Financial Strength** | Return on Equity | `info['returnOnEquity']` | Profitability per unit equity. |
| | Debt-to-Equity | `info['debtToEquity']` | Capital structure risk. |
| | Current Ratio | `info['currentRatio']` | Short-term solvency. |
| | Quick Ratio | `info['quickRatio']` | Acid-test liquidity. |
| | Oper. Cash Flow | `info['operatingCashflow']` | Cash backing earnings. |
| | Total Revenue | `info['totalRevenue']` | Company scale. |
| | EBITDA Margin | `info['ebitdaMargins']` | Core operational profitability. |
| **Growth Potential** | Revenue Growth | YoY revenue trend (derived) | Top-line expansion. |
| | Operating Margin | `info['operatingMargins']` | Efficiency scaling. |
| | Gross Margin | `info['grossMargins']` | Core pricing power. |
| | R&D to Revenue Ratio | `info['researchDevelopment'] / info['totalRevenue']` | Innovation intensity. |
| | Analyst Rating | `info['recommendationMean']` | External sentiment. |
| **Market Performance** | P/B Ratio | `info['priceToBook']` | Asset-based valuation. |
| | EPS | `info['trailingEps']` | Profit per share. |
| | Market Cap | `info['marketCap']` | Size proxy. |
| | EV/EBITDA | `info['enterpriseToEbitda']` | Debt-adjusted valuation. |
| | Price to Sales | `info['priceToSalesTrailing12Months']` | Revenue-based valuation. |
| | Dividend Payout Ratio | `info['payoutRatio']` | Earnings distribution signal. |
| **Risk & Volatility** | 30-d Hist. Volatility | 30-day std of daily returns | Short-term risk. |
| | Beta | `info['beta']` | Market sensitivity. |
| | Sharpe Ratio | excess return / vol (derived) | Risk-adjusted performance. |
| | Max Drawdown | roll-min on cumulative returns | Tail risk. |
| | Std Dev Returns | std of 1Y daily returns | Dispersion measure. |
| | Value at Risk (95%) | custom 5% percentile return | Downside tail exposure. |
| **Liquidity & Trading** | Avg 30-d Volume | rolling 30-day mean | Depth of book. |
| | Bid-Ask Spread | `(High−Low)/Mid` | Transaction cost proxy. |
| | Volume Growth | pct_change of volume | Activity momentum. |
| | Shares Outstanding | `info['sharesOutstanding']` | Supply side. |
| | Float Shares | `info['floatShares']` | Free float liquidity. |
| | Turnover Ratio | Volume / Float (custom) | Trading velocity. |

In [9]:
# This new indicator list drops: earnings_growth, pe_ratio and dividend_yield
# and adds: total_revenue, ebitda_margin, rnd_to_revenue, ev_to_ebitda, price_to_sales, payout_ratio and turnover_ratio

import json, os, datetime as dt
import pathlib

DOCS_DIR = pathlib.Path("..") / "docs"
DOCS_DIR.mkdir(parents=True, exist_ok=True)

METADATA_DIR = DOCS_DIR / "metadata"
METADATA_DIR.mkdir(parents=True, exist_ok=True)

base = "https://query2.finance.yahoo.com/v10/finance/quoteSummary"
today = dt.date.today().isoformat()

def yurl(ticker, module):
    return f"{base}/{ticker}?modules={module}"

var_dict = {
    # Financial Strength
    "roe": {
        "dim": "financial_strength",
        "definition": "Net income divided by total shareholder equity.",
        "unit": "%",
        "source_url": yurl("<ticker>", "defaultKeyStatistics"),
        "refresh": "quarterly",
        "field": "returnOnEquity",
        "last_updated": today
    },
    "debt_to_equity": {
        "dim": "financial_strength",
        "definition": "Total debt divided by shareholder equity.",
        "unit": "%",
        "source_url": yurl("<ticker>", "defaultKeyStatistics"),
        "refresh": "quarterly",
        "field": "debtToEquity",
        "last_updated": today
    },
    "current_ratio": {
        "dim": "financial_strength",
        "definition": "Current assets divided by current liabilities.",
        "unit": "ratio",
        "source_url": yurl("<ticker>", "financialData"),
        "refresh": "quarterly",
        "field": "currentRatio",
        "last_updated": today
    },
    "quick_ratio": {
        "dim": "financial_strength",
        "definition": "Liquid assets divided by current liabilities.",
        "unit": "ratio",
        "source_url": yurl("<ticker>", "financialData"),
        "refresh": "quarterly",
        "field": "quickRatio",
        "last_updated": today
    },
    "oper_cash_flow": {
        "dim": "financial_strength",
        "definition": "Cash generated by core operations.",
        "unit": "USD",
        "source_url": yurl("<ticker>", "cashflowStatementHistoryQuarterly"),
        "refresh": "quarterly",
        "field": "operatingCashflow",
        "last_updated": today
    },
    "total_revenue": {
        "dim": "financial_strength",
        "definition": "Total revenue reported by the company.",
        "unit": "USD",
        "source_url": yurl("<ticker>", "incomeStatementHistoryQuarterly"),
        "refresh": "quarterly",
        "field": "totalRevenue",
        "last_updated": today
    },
    "ebitda_margin": {
        "dim": "financial_strength",
        "definition": "EBITDA divided by total revenue.",
        "unit": "%",
        "source_url": yurl("<ticker>", "financialData"),
        "refresh": "quarterly",
        "field": "ebitdaMargins",
        "last_updated": today
    },

    # Growth Potential
    "revenue_growth": {
        "dim": "growth_potential",
        "definition": "Year-over-year percentage change in revenue.",
        "unit": "%",
        "source_url": yurl("<ticker>", "incomeStatementHistoryQuarterly"),
        "refresh": "quarterly",
        "field": "derived",
        "last_updated": today
    },
    "operating_margin": {
        "dim": "growth_potential",
        "definition": "Operating income divided by revenue.",
        "unit": "%",
        "source_url": yurl("<ticker>", "financialData"),
        "refresh": "quarterly",
        "field": "operatingMargins",
        "last_updated": today
    },
    "gross_margin": {
        "dim": "growth_potential",
        "definition": "Gross profit divided by revenue.",
        "unit": "%",
        "source_url": yurl("<ticker>", "financialData"),
        "refresh": "quarterly",
        "field": "grossMargins",
        "last_updated": today
    },
    "rnd_to_revenue": {
        "dim": "growth_potential",
        "definition": "R&D expenditure divided by total revenue.",
        "unit": "%",
        "source_url": yurl("<ticker>", "incomeStatementHistoryQuarterly"),
        "refresh": "quarterly",
        "field": "derived",
        "last_updated": today
    },
    "analyst_rating": {
        "dim": "growth_potential",
        "definition": "Wall-Street consensus rating (1 = Strong Bu, 5 = Sell).",
        "unit": "score",
        "source_url": yurl("<ticker>", "recommendationTrend"),
        "refresh": "weekly",
        "field": "recommendationMean",
        "last_updated": today
    },

    # Market Performance
    "pb_ratio": {
        "dim": "market_performance",
        "definition": "Share price divided by book value per share.",
        "unit": "ratio",
        "source_url": yurl("<ticker>", "defaultKeyStatistics"),
        "refresh": "daily",
        "field": "priceToBook",
        "last_updated": today
    },
    "eps": {
        "dim": "market_performance",
        "definition": "Trailing twelve-month earnings per share.",
        "unit": "USD",
        "source_url": yurl("<ticker>", "defaultKeyStatistics"),
        "refresh": "quarterly",
        "field": "trailingEps",
        "last_updated": today
    },
    "market_cap": {
        "dim": "market_performance",
        "definition": "Share price multiplied by shares outstanding.",
        "unit": "USD",
        "source_url": yurl("<ticker>", "price"),
        "refresh": "daily",
        "field": "marketCap",
        "last_updated": today
    },
    "ev_to_ebitda": {
        "dim": "market_performance",
        "definition": "Enterprise Value divided by EBITDA.",
        "unit": "ratio",
        "source_url": yurl("<ticker>", "defaultKeyStatistics"),
        "refresh": "quarterly",
        "field": "enterpriseToEbitda",
        "last_updated": today
    },
    "price_to_sales": {
        "dim": "market_performance",
        "definition": "Market cap divided by trailing 12-month sales.",
        "unit": "ratio",
        "source_url": yurl("<ticker>", "summaryDetail"),
        "refresh": "quarterly",
        "field": "priceToSalesTrailing12Months",
        "last_updated": today
    },
    "payout_ratio": {
        "dim": "market_performance",
        "definition": "Proportion of earnings paid as dividends.",
        "unit": "%",
        "source_url": yurl("<ticker>", "summaryDetail"),
        "refresh": "quarterly",
        "field": "payoutRatio",
        "last_updated": today
    },

    # Risk & Volatility
    "hist_volatility": {
        "dim": "risk_volatility",
        "definition": "30-day annualised volatility.",
        "unit": "%",
        "source_url": "price_history",
        "refresh": "daily",
        "field": "derived",
        "last_updated": today
    },
    "beta": {
        "dim": "risk_volatility",
        "definition": "Covariance with S&P 500 divided by variance of S&P 500.",
        "unit": "ratio",
        "source_url": yurl("<ticker>", "defaultKeyStatistics"),
        "refresh": "daily",
        "field": "beta",
        "last_updated": today
    },
    "sharpe_ratio": {
        "dim": "risk_volatility",
        "definition": "Excess return per unit of volatility.",
        "unit": "ratio",
        "source_url": "price_history",
        "refresh": "daily",
        "field": "derived",
        "last_updated": today
    },
    "max_drawdown": {
        "dim": "risk_volatility",
        "definition": "Largest peak-to-trough decline over past year.",
        "unit": "%",
        "source_url": "price_history",
        "refresh": "daily",
        "field": "derived",
        "last_updated": today
    },
    "stddev_returns": {
        "dim": "risk_volatility",
        "definition": "Standard deviation of daily returns over 252 days.",
        "unit": "%",
        "source_url": "price_history",
        "refresh": "daily",
        "field": "derived",
        "last_updated": today
    },
    "value_at_risk": {
        "dim": "risk_volatility",
        "definition": "95% Value at Risk over 1-day horizon.",
        "unit": "%",
        "source_url": "price_history",
        "refresh": "daily",
        "field": "derived",
        "last_updated": today
    },

    # Liquidity & Trading
    "avg_volume_30d": {
        "dim": "liquidity_trading",
        "definition": "30-day average daily trading volume.",
        "unit": "shares",
        "source_url": "price_history",
        "refresh": "daily",
        "field": "derived",
        "last_updated": today
    },
    "bid_ask_spread": {
        "dim": "liquidity_trading",
        "definition": "Relative bid-ask spread proxy.",
        "unit": "%",
        "source_url": "price_history",
        "refresh": "daily",
        "field": "derived",
        "last_updated": today
    },
    "volume_growth": {
        "dim": "liquidity_trading",
        "definition": "Monthly change in trading volume.",
        "unit": "%",
        "source_url": "price_history",
        "refresh": "daily",
        "field": "derived",
        "last_updated": today
    },
    "shares_outstanding": {
        "dim": "liquidity_trading",
        "definition": "Total company shares issued.",
        "unit": "shares",
        "source_url": yurl("<ticker>", "defaultKeyStatistics"),
        "refresh": "quarterly",
        "field": "sharesOutstanding",
        "last_updated": today
    },
    "float_shares": {
        "dim": "liquidity_trading",
        "definition": "Publicly tradable shares (excluding insiders).",
        "unit": "shares",
        "source_url": yurl("<ticker>", "defaultKeyStatistics"),
        "refresh": "quarterly",
        "field": "floatShares",
        "last_updated": today
    },
    "turnover_ratio": {
        "dim": "liquidity_trading",
        "definition": "30-day average volume divided by float.",
        "unit": "%",
        "source_url": "price_history",
        "refresh": "daily",
        "field": "derived",
        "last_updated": today
    }
}

with open(METADATA_DIR / "variable_dictionary.json", "w") as fp:
    json.dump(var_dict, fp, indent=2)

print("variable_dictionary.json written with full metadata.")


variable_dictionary.json written with full metadata.


## 2.7. Outputs Summary

* `universe.parquet` — 24-month, liquid company list  
* `variable_dictionary.json` — public metadata  
* `info_*.pkl` and `prices/*.parquet` — raw caches  

## 2.8. References

* Organisation for Economic Co-operation and Development: *Handbook on Constructing Composite Indicators: Methodology and User Guide*, 2008.
* Gao, Z. & Ongena, S. (2021). *Liquidity thresholds in U.S. equity markets*. *Journal of Financial Markets*.
* Greg Bland - yfinance library - https://algotrading101.com/learn/yfinance-guide/.
* GeeksforGeeks - yfinance library - https://www.geeksforgeeks.org/how-to-use-yfinance-api-with-python/.